In [1]:
import girder_client
%load_ext autoreload
%autoreload 2

import pandas as pd
import adrcSchemaHelpers as hlprs

gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")

Unknown Stain Tags: []
Unmatched File Names: []


In [2]:
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '638e2ae7cd1d7fad17b9dd1f',
 '_modelType': 'user',
 'admin': True,
 'created': '2022-12-05T17:31:19.547000+00:00',
 'email': 'admin@nowhere.nil',
 'emailVerified': True,
 'firstName': 'Admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'Admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 0,
 'status': 'enabled'}

In [ ]:
folders = [
    folder
    for folder in gc.listFolder('638e2da11f75016b81fda12f', parentFolderType='collection')
]


  #  _id: internal mongo id
  #  _modelType: object type (folder, item, or collection -- also annotations but not so relevant)
  #  baseParentId: collection the item is in
  #  baseParentType: parent object type (folder, item, or collection -- also annotations but not so relevant)
  #  created: date created
  #  creatorId: id of individual who created the item
  #  description: description of the item
  #  meta: dictionary of meta data pertaining to item
  #  name: item name (may be a year, patient id, or other folder/file names)
  #  parentCollection: *unclear, but told to ignore*
  #  parentId: id of parent collection
  #  public: whether item is public or not
  #  size: size in bytes (or other unit) -- folders show as 0
  #  updated: datetime of last update


In [ ]:
baseURL = "resource/*/items?type=folder&limit=1000"

itemSet = [
    item["meta"]["npSchema"]
    for folder in folders
    for subFolder in gc.listFolder(folder["_id"])
    for item in gc.getResource(baseURL.replace("*", subFolder['_id']))
    if item["meta"].get("npSchema") is not None
]

# So just to confuse things, I don't necessarily have to have a single folder below each case Folder
# To make things easy, I can just get all the items though, as I don't really care about what sub sub folder things are in
# See here
# http://candygram.neurology.emory.edu:8080/api/v1#!/resource/resource_getResourceItems_id_items


In [ ]:
allVals = {}

for item in itemSet:
    for (key, val) in item.items():
        if allVals.get(key) is None:
            allVals[key] = [val]
        elif val not in allVals[key]:
            allVals[key].append(val)

In [ ]:
lens = [len(val) for val in allVals.values()]
lens = [max(lens) - length for length in lens]

In [ ]:
# for pandas to make df from dict, columns must be of equal length, so adding "val" number of Nones
for key, val in zip(allVals.keys(), lens):
    if val:
        allVals[key].extend([None] * val)

In [ ]:
df = pd.DataFrame.from_dict(allVals)
df["stainMap"] = None

In [ ]:
stainList = ['ptdp', 'he', 'abeta', 'ubiq', "tau", "biels", "syn", "p62", "lfb"]
filt = (~df["stainID"].isin(stainList)) & (df["stainID"].notna())

df.loc[filt, "stainMap"] = df.loc[filt, "stainID"].map((lambda x: any([x.lower() in val for val in stainList])))

In [ ]:
filt = df["stainMap"].notna()
mappable = df.loc[filt, "stainID"].unique().tolist()

mappable = {key: val for key in mappable for val in stainList if val in key.lower()}
df["stainID"].replace(mappable, inplace=True)

In [ ]:
df.drop(columns=["stainMap"], inplace=True)

for col in df.columns:
    counted = df[col].value_counts()
    counted.name = "Count"
    counted.index.name = col
    counted.to_csv(f"./{col}_vals_counted.csv")

In [ ]:
df.drop_duplicates(subset=["stainID"], inplace=True)

df.to_csv("./all_vals.csv", index=False)

In [5]:
sampPatient = '638e33521f75016b81fdb8d2'  ## 2018 / E18-07

Unknown Stain Tags: []
Unmatched File Names: []


In [7]:
hlprs.scanMetadata(gc,{'_id':sampPatient},updateGirder=True)

Unknown Stain Tags: []
Unmatched File Names: []
E18-7_CON_AB.svs


NameError: name 'quit' is not defined